In [1]:
# Step 1: Import Necessary Libraries
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import nibabel as nib
from nilearn.connectome import ConnectivityMeasure
import matplotlib.pyplot as plt
from scipy.ndimage import center_of_mass
from sklearn.model_selection import train_test_split


2024-11-20 00:12:41.279380: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-20 00:12:41.286233: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732083161.294279   18518 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732083161.296699   18518 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-20 00:12:41.305152: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
os.environ['TF_GPU_ALLOCATOR']='cuda_malloc_async'

keras.backend.clear_session()

In [3]:
# Set up environment for GPU allocation
#os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_visible_devices(gpus[1], 'GPU')
        print(f"Using only GPU: {gpus[1]}")
    except RuntimeError as e:
        print(e)
else:
    print("No GPUs found.")

strategy = tf.distribute.OneDeviceStrategy(device="/gpu:1")


Using only GPU: PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')


I0000 00:00:1732083162.460361   18518 gpu_process_state.cc:201] Using CUDA malloc Async allocator for GPU: 1
I0000 00:00:1732083162.468515   18518 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 18438 MB memory:  -> device: 1, name: NVIDIA RTX 4000 Ada Generation, pci bus id: 0000:02:00.0, compute capability: 8.9


### TODO: have the subject_ids be read dynamically from the tsv file

In [4]:
# Step 2: Define Paths and Subject IDs

data_dir = '/home/zaz22/research-data-store/fmri/fmri_beijing'
mapping_file = '/home/zaz22/research-data-store/rois/rois_3000_beijing/rois/brain_atoms.mnc.gz'
phenotypic_file = '/home/zaz22/repos/adhd-mri-using-ml/adhd200_preprocessed_phenotypics.tsv'

subject_ids = ['9640133', '9783279', '9887336', '9890726', '4095748', '4136226', '4221029',
    '4225073', '4241194', '4256491', '4265987', '4334113', '4383707', '4475709',
    '4921428', '5150328', '5193577', '5575344', '5600820', '5669389', '5993008',
    '6187322', '6383713', '6477085', '6500128', '7011503', '7093319', '7135128',
    '7253183', '7390867', '7407032', '7689953', '7994085', '8191384', '8278680',]

# subject_ids = [
#     '9640133', '9783279', '9887336', '9890726', '4095748', '4136226', '4221029',
#     '4225073', '4241194', '4256491', '4265987', '4334113', '4383707', '4475709',
#     '4921428', '5150328', '5193577', '5575344', '5600820', '5669389', '5993008',
#     '6187322', '6383713', '6477085', '6500128', '7011503', '7093319', '7135128',
#     '7253183', '7390867', '7407032', '7689953', '7994085', '8191384', '8278680',
#     '8328877', '8838009', '9002207', '9093997', '9210521', '9221927', '9578631',
#     '3494778', '3554582', '3561920', '3562883', '3587000', '3593327', '3610134',
#     '3624598', '3655623', '3672300', '3672854', '3691107', '3707771', '3712305',
#     '3732101', '3739175', '3803759', '3809753', '3827352', '3856956', '3870624',
#     '3889095', '3910672', '3930512', '3967265', '3976121', '3983607', '3993793',
#     '3994098', '4006710', '4028266', '4048810', '4053388', '4053836', '4055710',
#     '4073815', '4075719', '4091983', '2538839', '2559537', '2601519', '2659769',
#     '2697768', '2703336', '2714224', '2737106', '2780647', '2833684', '2884672',
#     '2897046', '2907951', '2910270', '2919220', '2940712', '2950754', '2984158',
#     '3004580', '3086074', '3157406', '3194757', '3205761', '3212536', '3224401',
#     '3233028', '3239413', '3248920', '3262042', '3269608', '3277313', '3291029',
#     '3306863', '3308331', '3385520', '3390312', '3446674', '3473830', '1809715',
#     '1843546', '1860323', '1875013', '1875711', '1879542', '1912810', '1916266',
#     '1947991', '2031422', '2033178', '2081754', '2106109', '2107404', '2123983',
#     '2140063', '2141250', '2174595', '2196753', '2207418', '2208591', '2228148',
#     '2240562', '2249443', '2266806', '2268253', '2276801', '2296326', '2310449',
#     '2367157', '2377207', '2408774', '2427408', '2493190', '2498847', '2524687',
#     '2529026', '2535087', '1050345', '1050975', '1056121', '1068505', '1093743',
#     '1094669', '1113498', '1117299', '1132854', '1133221', '1139030', '1159908',
#     '1177160', '1186237', '1240299', '1258069', '1282248', '1302449', '1341865',
#     '1356553', '1391181', '1399863', '1404738', '1408093', '1411536', '1469171',
#     '1494102', '1561488', '1562298', '1628610', '1643780', '1662160', '1686092',
#     '1689948', '1771270', '1791543', '1794770', '1805037'
# ]



In [5]:
# Step 3: Load Functional Images and Extract Time Series for All Subjects

# Load the ROI mapping image (we'll use this to extract time series)
mapping_img = nib.load(mapping_file)
mapping_data = mapping_img.get_fdata()
affine = mapping_img.affine

# Get the unique labels (excluding 0)
roi_labels = np.unique(mapping_data)
roi_labels = roi_labels[roi_labels != 0]
n_rois = len(roi_labels)

print(f'Number of ROIs: {n_rois}')

# Create a masker for extracting time series from ROIs
from nilearn.maskers import NiftiLabelsMasker

labels_masker = NiftiLabelsMasker(
    labels_img=mapping_img,
    standardize=True,
    detrend=True,
    t_r=2.0,  # Replace with your TR (repetition time)
    verbose=1
)

# Initialize a list to store time series for all subjects
all_tseries = []

for sub_id in subject_ids:
    # data/fmri/beijing/fmri_X_1050345_session_1_run1.nii.gz
    func_file = os.path.join(data_dir, f'fmri_X_{sub_id}_session_1_run1.nii.gz')
    print(f'Loading functional image for {sub_id} from {func_file}')
    
    # Load the functional image
    func_img = nib.load(func_file)
    
    # Extract time series using the labels masker
    tseries = labels_masker.fit_transform(func_img)
    
    all_tseries.append(tseries)
    
    print(f'Extracted time series for {sub_id} with shape {tseries.shape}')


Number of ROIs: 2843
Loading functional image for 9640133 from /home/zaz22/research-data-store/fmri/fmri_beijing/fmri_X_9640133_session_1_run1.nii.gz
[NiftiLabelsMasker.wrapped] loading data from Minc1Image('/home/zaz22/research-data-store/rois/rois_3000_beijing/rois/brain_atoms.mnc.gz')
Resampling labels
[NiftiLabelsMasker.transform_single_imgs] Loading data from Nifti1Image('/home/zaz22/research-data-store/fmri/fmri_beijing/fmri_X_9640133_session_1_run1.nii.gz')
[NiftiLabelsMasker.transform_single_imgs] Extracting region signals
[NiftiLabelsMasker.transform_single_imgs] Cleaning extracted signals
Extracted time series for 9640133 with shape (236, 2843)
Loading functional image for 9783279 from /home/zaz22/research-data-store/fmri/fmri_beijing/fmri_X_9783279_session_1_run1.nii.gz
[NiftiLabelsMasker.wrapped] loading data from Minc1Image('/home/zaz22/research-data-store/rois/rois_3000_beijing/rois/brain_atoms.mnc.gz')
[NiftiLabelsMasker.transform_single_imgs] Loading data from Nifti1Ima

In [6]:
# Step 4: Extract ROI Coordinates (Same as Before)

# Extract ROI coordinates (center of mass)
roi_coords = []

for label in roi_labels:
    # Create a binary mask for the current ROI
    roi_mask = mapping_data == label

    # Calculate the center of mass in voxel space
    com_voxel = center_of_mass(roi_mask)

    # Convert voxel coordinates to world coordinates using the affine matrix
    com_world = nib.affines.apply_affine(affine, com_voxel)

    roi_coords.append(com_world)


In [7]:
# Step 5: Verify Alignment Between Time Series and ROI Labels

n_subjects = len(all_tseries)

# Check that all time series have the same number of ROIs
for idx, tseries in enumerate(all_tseries):
    assert tseries.shape[1] == n_rois, f"Mismatch in number of ROIs for subject {subject_ids[idx]}"

print(f'Number of subjects: {n_subjects}')
print(f'Number of ROIs in time series data: {n_rois}')

Number of subjects: 35
Number of ROIs in time series data: 2843


In [8]:
# Step 6: Compute Functional Connectivity Matrices for All Subjects
# import seaborn as sns

# Initialize the connectivity measure (using Pearson correlation)
correlation_measure = ConnectivityMeasure(kind='correlation')

# Compute correlation matrices
all_correlation_matrices = correlation_measure.fit_transform(all_tseries)

# for x in range(len(all_correlation_matrices)):
#     correlation_matrix = all_correlation_matrices[x]

#     # Create a figure and axis
#     plt.figure(figsize=(5,4))

#     # Plot the heatmap using seaborn
#     sns.heatmap(correlation_matrix, vmin=-1, vmax=1, cmap='coolwarm', square=True, cbar=True)

#     # Add title and labels
#     plt.title(f'Subject {x+1} Functional Connectivity Matrix')
#     plt.xlabel('ROIs')
#     plt.ylabel('ROIs')

#     # Show the plot
#     plt.show()

In [9]:
# Step 7: Prepare Group Labels

# Load phenotypic data
phenotypic = pd.read_csv(phenotypic_file, sep='\t')

# make subject ids into ints
int_subject_ids = [int(x) for x in subject_ids]

# Ensure the subject IDs match
phenotypic = phenotypic.set_index('ScanDir ID').loc[int_subject_ids].reset_index()

group_labels = phenotypic['DX'].values

# Check the distribution of group labels
print('Group label counts:')
print(pd.Series(group_labels).value_counts())


Group label counts:
0    21
3    10
1     4
Name: count, dtype: int64


### Diagnosis
- 0 Typically Developing Children
- 1 ADHD-Combined
- 2 ADHD-Hyperactive/Impulsive
- 3 ADHD-Inattentive

### Ideas

- combine all adhd classes into 1 and non-adhd to 0. Having more than 1 class of the adhd diagnosis is more difficult to diagnose from the cnn rather than just having 1 total ADHD class

- we could trim ROIS down in future versions. [Paper discussing this](https://pubmed.ncbi.nlm.nih.gov/38476041/)
![Alt text](/Users/user/Documents/GitHub/adhd-mri-using-ml/Subcortical-and-cortical-Regions-of-Interest-ROIs-Subcortical-ROIs-1a-Amy-amygdala.jpg "a title")

In [ ]:
X = np.array(all_correlation_matrices)# Use cross-validation for evaluation
kf = KFold(n_splits=5)
scores = []

for train_idx, test_idx in kf.split(conditions):
    decoder.fit(image.index_img(fmri_imgs, train_idx), conditions[train_idx])
    preds = decoder.predict(image.index_img(fmri_imgs, test_idx))
    accuracy = (preds == conditions[test_idx]).mean()
    scores.append(accuracy)

print(f"Cross-validation Accuracy: {np.mean(scores):.2f}")


for label in group_labels:
    if (label == '3'):
        label = '1'
y = group_labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
print(f'Data shape: {X.shape}')
print(f'Labels shape: {y.shape}')

X_train = X_train[..., np.newaxis]  # Shape: (n_samples, n_rois, n_rois, 1)
X_test = X_test[..., np.newaxis]

print(f'New Data shape: {X.shape}')
print(f'New Labels shape: {y.shape}')

Data shape: (35, 2843, 2843)
Labels shape: (35,)
New Data shape: (35, 2843, 2843)
New Labels shape: (35,)


In [12]:
print(f'X_train data type: {X_train.dtype}')
print(f'y_train data type: {y_train.dtype}')

X_train = X_train.astype(float)
y_train = y_train.astype(float)

print(f'new X_train data type: {X_train.dtype}')
print(f'new y_train data type: {y_train.dtype}')


X_train data type: float32
y_train data type: object
new X_train data type: float64
new y_train data type: float64


In [ ]:
from keras import layers, models

with strategy.scope():
    model = models.Sequential([
        layers.Input(shape=(2843,2843,1)),
        layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='linear')
    ])
    opt = tf.keras.optimizers.Adam(learning_rate=0.01)
    model.compile(optimizer=opt, loss='mean_squared_error', metrics=['accuracy'])

# Adjust batch size dynamically
batch_size = 4
while True:
    try:
        history = model.fit(X_train, y_train, epochs=5, batch_size=batch_size, validation_split=0.2)
        break
    except tf.errors.ResourceExhaustedError:
        batch_size //= 2
        if batch_size == 0:
            raise ValueError("Not enough GPU memory to process the data with current settings.")
        print(f"Reducing batch size to {batch_size}")


2024-11-20 00:13:45.644223: E external/local_xla/xla/stream_executor/gpu/gpu_cudamallocasync_allocator.cc:359] gpu_async_1 cuMemAllocAsync failed to allocate 8235925504 bytes: RESOURCE_EXHAUSTED: : CUDA_ERROR_OUT_OF_MEMORY: out of memory
 Reported by CUDA: Free memory/Total memory: 1615462400/21125267456
2024-11-20 00:13:45.644244: E external/local_xla/xla/stream_executor/gpu/gpu_cudamallocasync_allocator.cc:364] Stats: Limit:                     19334299648
InUse:                     16471927384
MaxInUse:                  16471927388
NumAllocs:                          38
MaxAllocSize:               8235925504
Reserved:                            0
PeakReserved:                        0
LargestFreeBlock:                    0

2024-11-20 00:13:45.644249: E external/local_xla/xla/stream_executor/gpu/gpu_cudamallocasync_allocator.cc:68] Histogram of current allocation: (allocation_size_in_bytes, nb_allocation_of_that_sizes), ...;
2024-11-20 00:13:45.644252: E external/local_xla/xla/strea

ResourceExhaustedError: {{function_node __wrapped__AddV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:AddV2] name: 

In [ ]:
X_test = X_test.astype(float)
y_test = y_test.astype(float)

test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()